# Using satellite imagery to create near-real-time flood maps
---

If you did not create a virtual environmnet, uncomment this sections to install python modules.

In [ ]:
# # Install Modules
# !pip install requests
# !pip install geopandas
# !pip install pandas


In [10]:
import requests
import geopandas as gpd
import pandas as pd
from scripts.create_sqlitedb import create_db

In [11]:
# Create a database
db = create_db()  # This will prompt for input

Database already exists


To facilitate the API URL, we need to get a FIPS code for our Kentucky Counties [KyGISServer](https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_Counties_Generalized_WGS84WM/MapServer/0) Rest Endpoint.



In [12]:
# Pull in County Data Information
counties_url = 'https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_Counties_Generalized_WGS84WM/MapServer/0/query?where=1%3D1&units=esriSRUnit_Foot&outFields=NAME2%2CFIPS_ID&returnExtentOnly=false&featureEncoding=esriDefault&f=pjson'
df = gpd.read_file(counties_url)
print(df.dtypes)
df.head(3)

NAME2         object
FIPS_ID      float64
geometry    geometry
dtype: object


,NAME2,FIPS_ID,geometry
0,Greenup,21089.0,"POLYGON ((-9242952.878 4682431.15, -9241037.90..."
1,Mason,21161.0,"POLYGON ((-9340139.537 4688519.829, -9335101.3..."
2,Carroll,21041.0,"POLYGON ((-9499237.108 4683958.597, -9494357.7..."


\*The FIPS_ID is a float, we can worry about that later if it's important.  Might be easier to leave as is and manipulated with SQL.

---

Use USGS [StreamStats (API) Service](https://waterservices.usgs.gov/test-tools/?service=iv&siteType=&statTypeCd=all&major-filters=state-territory&stateCd=ky&format=json&date-type=type-absolute&statReportType=daily&statYearType=calendar&missingData=off&siteStatus=all&siteNameMatchOperator=start) to query water level data.

Set up a Sqlite database with the spatialite extension -> [setup documentation](https://til.simonwillison.net/spatialite/minimal-spatialite-database-in-python)